In [ ]:
import os
import json
import pandas as pd
import traceback


In [ ]:
!pip install langchain-community

In [ ]:
from langchain.chat_models import ChatOpenAI

    

In [73]:
from dotenv import load_dotenv

load_dotenv()


True

In [74]:
KEY = os.getenv("OPENAI_API_KEY")


In [75]:
llm = ChatOpenAI(openai_api_key = KEY,model_name="gpt-3.5-turbo",temperature=0.5)

In [ ]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2
from langchain.chains import LLMChain


In [ ]:
RESPONSE_JSON = {
    "1": {
        "mcq" : "multiple choice question",
        "options": {
            "a" : "choice here",
            "b" : "choice here",
            "c" : "choice here",
            "d" : "choice here",
        },
        "correct" : "correct answer",

    },
    "2": {
        "mcq" : "multiple choice question",
        "options": {
            "a" : "choice here",
            "b" : "choice here",
            "c" : "choice here",
            "d" : "choice here",
        },
        "correct" : "correct answer",

    },
    "3": {
        "mcq" : "multiple choice question",
        "options": {
            "a" : "choice here",
            "b" : "choice here",
            "c" : "choice here",
            "d" : "choice here",
        },
        "correct" : "correct answer",

    },
    "4": {
        "mcq" : "multiple choice question",
        "options": {
            "a" : "choice here",
            "b" : "choice here",
            "c" : "choice here",
            "d" : "choice here",
        },
        "correct" : "correct answer",

    },"5": {
        "mcq" : "multiple choice question",
        "options": {
            "a" : "choice here",
            "b" : "choice here",
            "c" : "choice here",
            "d" : "choice here",
        },
        "correct" : "correct answer",

    },
    "6": {
        "mcq" : "multiple choice question",
        "options": {
            "a" : "choice here",
            "b" : "choice here",
            "c" : "choice here",
            "d" : "choice here",
        },
        "correct" : "correct answer",

    },
    "7": {
        "mcq" : "multiple choice question",
        "options": {
            "a" : "choice here",
            "b" : "choice here",
            "c" : "choice here",
            "d" : "choice here",
        },
        "correct" : "correct answer",

    },
    "8": {
        "mcq" : "multiple choice question",
        "options": {
            "a" : "choice here",
            "b" : "choice here",
            "c" : "choice here",
            "d" : "choice here",
        },
        "correct" : "correct answer",

    },
    "9": {
        "mcq" : "multiple choice question",
        "options": {
            "a" : "choice here",
            "b" : "choice here",
            "c" : "choice here",
            "d" : "choice here",
        },
        "correct" : "correct answer",

    },
    "10": {
        "mcq" : "multiple choice question",
        "options": {
            "a" : "choice here",
            "b" : "choice here",
            "c" : "choice here",
            "d" : "choice here",
        },
        "correct" : "correct answer",

    },

}

In [ ]:
TEMPLATE = """
Text :{text}
You are an expert MCQ maker. Given the above text , it is your job to \
create a Quiz of {number} multiple choice Questons for {subject} students in {tone} tone.
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like RESPONSE_JSON below and use it as a guide.\
insure to make {number} MCQs
### RESPONSE_JSON
{response_json}

"""

In [ ]:
quiz_generation_prompt = PromptTemplate(
    input_variables=["text", "number", "subject", "tone", "response_json"],
    template = TEMPLATE
)

In [ ]:
quiz_chain = LLMChain(llm = llm, prompt = quiz_generation_prompt, output_key = "quiz", verbose = True)


In [ ]:
TEMPLATE2 = """
You are and expert english grammerian and writer . Given a Multiple Choice for {subject} students.\
You read to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity
if the quiz is not at per the with the cognitive and analytical abilties of the students,\
update the quiz questions with needs to be changed and change the tone such that it perfectly fits the students.
Quiz_MCQs:
{quiz}

Check froma nd expert English writer of the above Quiz:
""" 

In [ ]:
quiz_evaluation_prompt = PromptTemplate(input_variables=["subject", "quiz"], template = TEMPLATE)

In [ ]:
review_chain = LLMChain(llm = llm, prompt = quiz_generation_prompt, output_key = "review", verbose = True)

In [ ]:
generate_evaluate_chain = SequentialChain(chains = [quiz_chain, review_chain], input_variables = ["text", "number", "subject", "tone", "response_json"],
                                           output_variables = ["quiz", "review"], verbose = True,)


In [ ]:
file_path =r"D:\new_for_project\mcqgen\data.txt"

In [ ]:
file_path

In [ ]:
with open(file_path, 'r') as file:

    TEXT = file.read()

In [ ]:
print(TEXT)

In [ ]:
#serialize the python dictionary into a json formatted string
json.dumps(RESPONSE_JSON)


In [ ]:
NUMBER = 4
SUBJECT = "machine learning"
TONE = "simple"


In [ ]:
#https://python.langchain.com/docs/modules/model_io/llms/token_usage_tracking

#How to setup Token Usage Tracking in LangChain
with get_openai_callback() as cb:
    response=generate_evaluate_chain(
        {
            "text": TEXT,
            "number": NUMBER,
            "subject":SUBJECT,
            "tone": TONE,
            "response_json": json.dumps(RESPONSE_JSON)
        }
    )

In [ ]:
print(f"Total Tokens:{cb.total_tokens}")
print(f"Prompt Tokens:{cb.prompt_tokens}")
print(f"Completion Tokens:{cb.completion_tokens}")
print(f"Total Cost:{cb.total_cost}")

In [ ]:
response

In [ ]:
response.get("quiz")

In [66]:
quiz = json.loads(quiz)

NameError: name 'quiz' is not defined

In [ ]:
quiz_table_data = []
for key, value in quiz.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
            ]
        )
    correct = value["correct"]
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})

In [67]:
quiz_table_data

[]

In [69]:
quiz = pd.DataFrame(quiz_table_data)

In [70]:
quiz.to_csv("machinelearning.csv",index=False)

In [71]:
from datetime import datetime
datetime.now().strftime('%m_%d_%Y_%H_%M_%S')

'12_01_2024_10_04_06'